In [ ]:
from typing import Optional
import os
import re
import time
import json

from openai import OpenAI, APIError, RateLimitError
from google.colab import userdata

In [ ]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError(
        "Colab 보안 비밀에 'OPENAI_API_KEY'가 설정되지 않았습니다. 키를 추가해주세요."
    )
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def call_llm_api(
    messages: list[dict[str, str]],
    client: OpenAI,
    temperature: float = 0.0,
    model: str = "gpt-4o",
    json_mode: bool = False,
) -> str | None:
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        response_format={
            "type": "json_object" if json_mode else "text"
        },
    )
    return completion.choices[0].message.content

In [ ]:
def generate_initial_response(
    client: Optional[OpenAI],
    question: str,
    model: str="gpt-4o",
    temperature: float=0.5,
) -> Optional[str]:
    """[단계 1] 사용자 질문에 대한 초기 답변 초안을 생성"""
    print(f"\n--- [단계 1] 초기 답변 생성 ---")
    print(f"질문: {question}")

    messages: list[dict[str, str]] = [{"role": "user", "content": question}]

    response: Optional[str] = call_llm_api(messages, client, temperature=temperature, model=model, json_mode=False)
    return response

In [ ]:
initial_question: str = "마리 퀴리가 노벨상을 수상한 분야와 그 업적은 무엇인가요? 간략히 설명해주세요."
step1_output: Optional[str] = generate_initial_response(client, initial_question)
step1_output


--- [단계 1] 초기 답변 생성 ---
질문: 마리 퀴리가 노벨상을 수상한 분야와 그 업적은 무엇인가요? 간략히 설명해주세요.


'마리 퀴리는 두 번의 노벨상을 수상한 과학자로, 각각 다른 분야에서 수상했습니다. \n\n1. **1903년 노벨 물리학상**: 마리 퀴리는 남편 피에르 퀴리, 그리고 앙리 베크렐과 함께 방사능 연구로 노벨 물리학상을 수상했습니다. 이들은 방사능 현상에 대한 연구를 통해 자연계에 존재하는 방사능의 특성을 밝혀냈습니다.\n\n2. **1911년 노벨 화학상**: 마리 퀴리는 라듐과 폴로늄을 발견하고, 라듐의 분리 및 특성을 연구한 공로로 노벨 화학상을 수상했습니다. 그녀의 연구는 방사성 원소의 화학적 성질을 이해하는 데 중요한 기여를 했습니다.\n\n마리 퀴리는 이러한 업적을 통해 방사능 연구의 선구자 역할을 했으며, 과학사에서 중요한 인물로 평가받고 있습니다.'

In [ ]:
# CoVe를 위해 의도적으로 step1_output을 할루시네이션 결과물로 대체.
# 아래 코드를 사용하지 않고, LLM의 결과가 저장된 step1_output을 바로 사용해도 되지만, 할루시네이션이 발생하지 않을 수도 있음.
step1_hypothetical_output: str = """마리 퀴리는 역사상 유일하게 서로 다른 두 과학 분야에서 노벨상을 수상한 위대한 과학자입니다. 그녀는 1903년에 남편 피에르 퀴리, 앙리 베크렐과 함께 방사선 연구에 대한 공로로 노벨 물리학상을 공동 수상했습니다. 이후 1911년에는 라듐과 폴로늄의 발견 및 순수 라듐 분리 연구로 노벨 화학상을 단독으로 수상했습니다. 또한, 제1차 세계 대전 중 이동식 X선 장치를 개발하여 부상병 치료에 기여한 공로로 노벨 평화상 후보에도 여러 차례 올랐으며, 사실상 수상자로 인정받고 있습니다."""
step1_output = step1_hypothetical_output
step1_output

'마리 퀴리는 역사상 유일하게 서로 다른 두 과학 분야에서 노벨상을 수상한 위대한 과학자입니다. 그녀는 1903년에 남편 피에르 퀴리, 앙리 베크렐과 함께 방사선 연구에 대한 공로로 노벨 물리학상을 공동 수상했습니다. 이후 1911년에는 라듐과 폴로늄의 발견 및 순수 라듐 분리 연구로 노벨 화학상을 단독으로 수상했습니다. 또한, 제1차 세계 대전 중 이동식 X선 장치를 개발하여 부상병 치료에 기여한 공로로 노벨 평화상 후보에도 여러 차례 올랐으며, 사실상 수상자로 인정받고 있습니다.'

In [ ]:
def plan_verifications_json(
    client: Optional[OpenAI],
    initial_response: str,
    model: str="gpt-4o",
    temperature: float=0.0,
) -> Optional[str]:
    """
    [단계 2] 초기 답변을 바탕으로 검증 질문 목록 JSON을 계획.

    Args:
        client: OpenAI 클라이언트 객체.
        initial_response: 단계 1에서 생성된 초기 답변 문자열.
        model: 사용할 LLM 모델 이름 (JSON 모드 지원 필요).
        temperature: 생성 결과의 무작위성 조절 값.

    Returns:
        {"verification_questions": ["질문1", "질문2", ...]} 형태의 딕셔너리 또는 실패 시 None.
    """
    print(f"\n--- [단계 2] 검증 계획 수립 요청 (JSON 출력) ---")
    plan_verification_prompt: str = f"""
당신이 생성한 이전 답변 초안은 다음과 같습니다:
{initial_response}

위 답변 초안의 내용을 주의 깊게 분석하여, 사실 확인이 필요한 주요 주장들을 식별하세요.
이러한 주장은 답변에 언급된 구체적인 인물, 사건, 날짜, 수치, 업적, 인과 관계 등을 포함할 수 있습니다.

식별된 각 주요 주장에 대해, 그 주장의 사실적 정확성을 객관적으로 검증하기 위한 구체적인 질문을 생성해야 합니다.

질문 생성 가이드라인:
- 각 질문은 초기 답변의 특정 주장을 명확히 타겟해야 합니다.
- 질문은 간결하고 명확해야 하며, 답변 가능한 형태여야 합니다.
- 초기 답변에 제시된 정보를 단순히 반복하는 질문이 아니라, 정보의 진위를 묻는 질문이어야 합니다.
- 필요하다면, 주장의 근거나 출처를 묻는 질문도 포함할 수 있습니다.

생성된 검증 질문 목록을 반드시 다음 JSON 형식으로 반환해주세요.
JSON 객체는 \"verification_questions\"라는 단일 키를 가져야 하며, 값은 질문 문자열들의 리스트여야 합니다.

예시:
{{
  "verification_questions": [
    "첫 번째 검증 질문 텍스트",
    "두 번째 검증 질문 텍스트",
    ...
  ]
}}
"""

    messages: list[dict[str, str]] = [{"role": "user", "content": plan_verification_prompt}]
    return call_llm_api(messages, client, temperature=temperature, model=model, json_mode=True)

In [ ]:
def parse_verification_questions_from_json(plan_output_json: Optional[str]) -> list[str]:
    """단계 2의 JSON 출력 문자열을 파싱하여 질문 리스트를 반환"""

    if not plan_output_json:
        print("[오류] 파싱할 검증 계획 JSON 출력이 없습니다.")
        return []

    try:
        verification_questions_dict = json.loads(plan_output_json)
        questions = [str(q) for q in verification_questions_dict["verification_questions"]]
        print(f"[정보] JSON에서 {len(questions)}개의 검증 질문을 성공적으로 파싱했습니다.")
        return questions
    except json.JSONDecodeError as e:
        print(f"[오류] JSON 파싱 실패: {e}. 원본 응답: {plan_output_json}")
        return []
    except KeyError:
        print(f"[오류] JSON 형식이 예상과 다릅니다: {plan_output_json}")
        return []
    except Exception as e:
        print(f"[오류] 검증 질문 파싱 중 예외 발생: {e}")
        return []

In [ ]:
# --- 단계 2 함수 호출 ---
verification_questions: list[str] = []
step2_output_json_str = plan_verifications_json(client, step1_output)

# --- 단계 2 결과 파싱 ---
verification_questions = parse_verification_questions_from_json(step2_output_json_str)
verification_questions


--- [단계 2] 검증 계획 수립 요청 (JSON 출력) ---
[정보] JSON에서 5개의 검증 질문을 성공적으로 파싱했습니다.


['마리 퀴리가 역사상 유일하게 서로 다른 두 과학 분야에서 노벨상을 수상한 과학자인가요?',
 '마리 퀴리가 1903년에 피에르 퀴리와 앙리 베크렐과 함께 노벨 물리학상을 수상한 것이 사실인가요?',
 '마리 퀴리가 1911년에 라듐과 폴로늄의 발견 및 순수 라듐 분리 연구로 노벨 화학상을 단독 수상했나요?',
 '마리 퀴리가 제1차 세계 대전 중 이동식 X선 장치를 개발하여 부상병 치료에 기여한 것이 사실인가요?',
 '마리 퀴리가 노벨 평화상 후보에 여러 차례 올랐으며, 사실상 수상자로 인정받았다는 주장의 근거는 무엇인가요?']

In [ ]:
def parse_qna_pairs_from_json(
    qna_output_json: Optional[str],
) -> Optional[list[dict[str, str]]]:
    """단계 3의 QnA JSON 출력 문자열을 파싱하여 QnA 딕셔너리 리스트를 반환"""
    if not qna_output_json:
        print("[오류] 파싱할 검증 실행 JSON 출력이 없습니다.")
        return None

    try:
        qna_output_dict = json.loads(qna_output_json)
        qna_list = qna_output_dict["qna_pairs"]
        print(f"[정보] JSON에서 {len(qna_list)}개의 QnA 쌍을 성공적으로 파싱했습니다.")
        return qna_list
    except json.JSONDecodeError as e:
        print(f"[오류] LLM 응답 JSON 파싱 실패: {e}. 원본 응답: {qna_output_json}")
        return None
    except KeyError:
        print(f"[오류] JSON 형식이 예상과 다릅니다: {qna_output_json}")
        return None
    except Exception as e:
        print(f"[오류] QnA 파싱 중 예외 발생: {e}")
        return None

In [ ]:
def execute_verifications_json(
    client: Optional[OpenAI],
    verification_questions: list[str],
    model: str = "gpt-4o",
    temperature: float = 0.0,
) -> Optional[str]:
    """
    [단계 3] 계획된 검증 질문 목록에 대해 독립적인 답변을 생성 및질문-답변 쌍 리스트를 포함하는 JSON 문자열 형식으로 반환.

    Args:
        client: OpenAI 클라이언트 객체.
        verification_questions: 검증 질문 문자열 리스트.
        model: 사용할 LLM 모델 이름 (JSON 모드 지원 필요).
        temperature: 생성 결과의 무작위성 조절 값.

    Returns:
        {"qna_pairs": [{"question": "...", "answer": "..."}, ...]} 형태의 JSON 문자열 또는 실패 시 None.
    """
    print(f"\n--- [단계 3] 검증 실행 요청 (JSON 출력) ---")
    if not verification_questions:
        print("[오류] 검증할 질문이 없습니다.")
        return None

    questions_formatted_for_prompt = "\n".join([f"{i+1}. {q}" for i, q in enumerate(verification_questions)])
    execute_verification_prompt: str = f"""다음은 검증이 필요한 질문 목록입니다:
{questions_formatted_for_prompt}

각 질문 번호에 맞춰, 독립적으로, 그리고 현재 알려진 역사적 사실에 기반하여 답변해주세요.
답변은 간결하고 명확해야 합니다. 정보가 확실하지 않다면 답변에 명시해주세요.

최종 결과는 반드시 다음 JSON 형식으로 반환해야 합니다. 'qna_pairs' 리스트에는 각 원본 질문과 그에 대한 답변을 포함하는 객체를 순서대로 넣어야 합니다.

예시:
{{
  "qna_pairs": [
    {{
      "question": "여기에 첫 번째 질문 텍스트",
      "answer": "여기에 첫 번째 질문에 대한 답변"
    }},
    {{
      "question": "여기에 두 번째 질문 텍스트",
      "answer": "여기에 두 번째 질문에 대한 답변"
    }},
    ...
  ]
}}
"""

    messages: list[dict[str, str]] = [{"role": "user", "content": execute_verification_prompt}]
    return call_llm_api(messages, client, temperature=temperature, model=model, json_mode=True)

In [ ]:
# --- 단계 3 함수 호출 ---
step3_output_json_str = execute_verifications_json(client, verification_questions)

# --- 단계 3 결과 파싱 ---
verification_qna_list = parse_qna_pairs_from_json(step3_output_json_str)
verification_qna_list


--- [단계 3] 검증 실행 요청 (JSON 출력) ---
[정보] JSON에서 5개의 QnA 쌍을 성공적으로 파싱했습니다.


[{'question': '마리 퀴리가 역사상 유일하게 서로 다른 두 과학 분야에서 노벨상을 수상한 과학자인가요?',
  'answer': '네, 마리 퀴리는 물리학과 화학에서 각각 노벨상을 수상한 유일한 과학자입니다.'},
 {'question': '마리 퀴리가 1903년에 피에르 퀴리와 앙리 베크렐과 함께 노벨 물리학상을 수상한 것이 사실인가요?',
  'answer': '네, 마리 퀴리는 1903년에 피에르 퀴리와 앙리 베크렐과 함께 노벨 물리학상을 수상했습니다.'},
 {'question': '마리 퀴리가 1911년에 라듐과 폴로늄의 발견 및 순수 라듐 분리 연구로 노벨 화학상을 단독 수상했나요?',
  'answer': '네, 마리 퀴리는 1911년에 라듐과 폴로늄의 발견 및 순수 라듐 분리 연구로 노벨 화학상을 단독 수상했습니다.'},
 {'question': '마리 퀴리가 제1차 세계 대전 중 이동식 X선 장치를 개발하여 부상병 치료에 기여한 것이 사실인가요?',
  'answer': '네, 마리 퀴리는 제1차 세계 대전 중 이동식 X선 장치를 개발하여 부상병 치료에 기여했습니다.'},
 {'question': '마리 퀴리가 노벨 평화상 후보에 여러 차례 올랐으며, 사실상 수상자로 인정받았다는 주장의 근거는 무엇인가요?',
  'answer': '마리 퀴리가 노벨 평화상 후보에 올랐다는 기록은 없습니다. 따라서 사실상 수상자로 인정받았다는 주장의 근거는 없습니다.'}]

In [ ]:
def generate_final_response_from_json(
    client: Optional[OpenAI],
    initial_response: str,
    verification_qna_list: list[dict[str, str]],
    model: str="gpt-4o",
    temperature: float=0.0,
) -> Optional[str]:
    """
    [단계 4] 초기 답변과 검증된 QnA 목록을 종합하여 최종 답변(텍스트)을 생성 및 수정.

    Args:
        client: OpenAI 클라이언트 객체.
        initial_response: 단계 1에서 생성된 초기 답변 문자열.
        verification_qna_list: 단계 3의 결과를 파싱한 검증 QnA 딕셔너리 리스트.
        model: 사용할 LLM 모델 이름.
        temperature: 생성 결과의 무작위성 조절 값.

    Returns:
        최종 검증 및 수정된 답변 문자열 또는 실패 시 None.
    """
    print(f"\n--- [단계 4] 최종 답변 생성/수정 요청 ---")
    verification_summary = "\n\n".join([
        f"질문: {item.get('question', 'N/A')}\n검증된 답변: {item.get('answer', 'N/A')}"
        for item in verification_qna_list
    ])

    final_response_prompt: str = f"""
당신의 초기 답변 초안은 다음과 같았습니다.
{initial_response}

그리고 위 답변을 검증한 결과는 다음과 같습니다 (질문-답변 쌍 목록).
{verification_summary}

이제 이 모든 정보(초기 답변, 검증 결과)를 종합적으로 검토하세요.
검증된 답변 내용을 바탕으로 초기 답변 초안의 오류(만약 있다면, 예를 들어 '노벨 평화상' 관련 내용)를 **반드시 수정**하고, 검증된 정확한 정보를 반영하여 마리 퀴리의 노벨상 수상 업적에 대한 **최종 답변을 자연스러운 문장으로 생성**해주세요.
초기 답변의 올바른 내용은 유지하되, 검증 결과와 다른 내용은 수정해야 합니다. 최종 답변에는 검증 질문이나 '검증 결과'라는 문구는 포함하지 말고, 최종적으로 사용자에게 전달할 완성된 설명만 작성해주세요.
""".strip()

    messages: list[dict[str, str]] = [{"role": "user", "content": final_response_prompt}]

    response: Optional[str] = call_llm_api(messages, client, temperature=temperature, model=model, json_mode=False)
    return response

In [ ]:
# --- 단계 4 함수 호출 ---
final_verified_output = generate_final_response_from_json(
    client,
    step1_output,
    verification_qna_list,
)
final_verified_output


--- [단계 4] 최종 답변 생성/수정 요청 ---


'마리 퀴리는 역사상 유일하게 서로 다른 두 과학 분야에서 노벨상을 수상한 위대한 과학자입니다. 그녀는 1903년에 남편 피에르 퀴리와 앙리 베크렐과 함께 방사선 연구에 대한 공로로 노벨 물리학상을 공동 수상했습니다. 이후 1911년에는 라듐과 폴로늄의 발견 및 순수 라듐 분리 연구로 노벨 화학상을 단독으로 수상했습니다. 또한, 제1차 세계 대전 중에는 이동식 X선 장치를 개발하여 부상병 치료에 기여했습니다. 그러나 마리 퀴리가 노벨 평화상 후보에 올랐다는 기록은 없습니다.'